In [1]:
import sys
sys.path.append('..')

from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from enum import Enum
from src.services import FacebookCreativeDescriptive
from src.interfaces.descriptive import facebook_creative_descriptive as fcd, facebook_target_descriptive as ftd
from src.services.facebook_image_descriptive_saver import facebook_image_descriptive_saver as fids
from src.interfaces.descriptive import google_campaign_type_descriptive as gcd
from src.database.session import SessionLocal
from src.pingers import *
from src.utils import *
from src import crud
from src.interfaces.descriptive import DescriptiveDF
from src.crud.utils.get_performance import get_performance, get_conversion_rates_dict
from src.statistical_tests import mean_test, mean_test_cr
db = SessionLocal()
end_date_plus_one = date(year=2023, month=8, day=1)
end_date = end_date_plus_one - timedelta(days=1)
start_date = end_date_plus_one - relativedelta(months=24)
shop_id = 2445081

2023-11-27 16:05:53.441 | DEBUG    | src.database.session:<module>:17 - base: prod.cw6tznt1p9hq.us-east-1.rds.amazonaws.com


In [2]:
df = fcd.read_df(DescriptiveDF.main, '2023-10-31')

2023-11-27 09:21:58.784 | DEBUG    | src.abc.s3_reader_writer:read_df:24 - ####: descriptive/facebook_creative/main_2023-10-31.csv
2023-11-27 09:21:58.785 | DEBUG    | src.utils.s3.read_file_from_s3:read_csv_from_s3:24 - path: dev/descriptive/facebook_creative/main_2023-10-31.csv


In [2]:
df = gcd.read_df(DescriptiveDF.main, df_id=gcd.end_date)

2023-11-27 16:05:55.046 | DEBUG    | src.abc.s3_reader_writer:read_df:24 - ####: descriptive/google_campaign_type/main_2023-10-31.csv
2023-11-27 16:05:55.047 | DEBUG    | src.utils.s3.read_file_from_s3:read_csv_from_s3:24 - path: dev/descriptive/google_campaign_type/main_2023-10-31.csv


In [7]:
max(df.year_month)

'2023-10'

In [3]:
metric = cpm
for metric in (cpm,):
    df[str(metric)] = df.apply(metric.formula_series, axis=1)

In [80]:
def split_series(series):
    total_sum = series.sum()

    left_sum = 0
    left_index = 0

    logger.debug(total_sum)

    for i, value in enumerate(series):
        left_sum += value
        if left_sum >= total_sum / 2:
            left_index = i
            logger.debug(left_sum)
            logger.debug(f'{total_sum / 2}')
            break

    return left_index + 1

df_by_shop = df.groupby('shop_id').agg({f'{cpm}': 'mean', 'spend_USD': 'sum'}).sort_values(f'{cpm}')
split_series_idx = split_series(df_by_shop['spend_USD'])
low_cpm_shop_ids = df_by_shop[:split_series_idx].index.tolist()
high_cpm_shop_ids = df_by_shop[split_series_idx:].index.tolist()

2023-11-27 10:02:26.052 | DEBUG    | __main__:split_series:7 - 5283627943.340359
2023-11-27 10:02:26.052 | DEBUG    | __main__:split_series:13 - 2696544968.2248483
2023-11-27 10:02:26.053 | DEBUG    | __main__:split_series:14 - 2641813971.6701794


In [95]:
feature = 'free_shipping'
feature_df = df[df.feature == feature]
sample_dict = {value: feature_df.loc[feature_df.feature_value == value, f'{metric}'].tolist() for value in feature_df.feature_value.unique()}
logger.debug(feature_df.groupby(feature_df.feature_value)[['spend_USD']].sum() / feature_df.spend_USD.sum())
test_result = mean_test(samples_dict=sample_dict)
test_result

2023-11-27 10:16:13.763 | DEBUG    | __main__:<module>:4 -                spend_USD
feature_value           
False           0.919341
True            0.080659


{'stat': 642.667011764255,
 'p': 8.788311499397401e-142,
 'test_type': 'kruskal',
 'stat_levene': 343.0666604140537,
 'p_levene': 3.728566312839868e-76,
 False: {'mean': 8.968443361887646, 'size': 23490},
 True: {'mean': 13.129344270829161, 'size': 5863}}

In [87]:
high_cpm_true_df = feature_df[
    (feature_df.shop_id.isin(high_cpm_shop_ids)) & (feature_df.feature_value == True)
]
low_cpm_true_df = feature_df[(feature_df.shop_id.isin(low_cpm_shop_ids)) & (feature_df.feature_value == True)]
logger.debug(high_cpm_true_df.spend_USD.sum())
logger.debug(low_cpm_true_df.spend_USD.sum())

2023-11-27 10:03:40.170 | DEBUG    | __main__:<module>:5 - 24826628.473471552
2023-11-27 10:03:40.171 | DEBUG    | __main__:<module>:6 - 15532202.746534597


In [59]:
df.loc[df.shop_id.isin(low_cpm_shop_ids), 'spend_USD'].sum()

77266606.52

In [37]:
joined_df = pd.DataFrame(true_top_df).join(false_top_df, how='outer').head(100)

In [38]:
joined_df.sum()

true     100.0
false    100.0
dtype: float64

In [20]:
pd.DataFrame(top_df, columns=['top']).join(pd.DataFrame(bottom_df, columns=['bottom']))

,top,bottom


In [2]:
df = fcd.get_shop_df(db=db, shop_id=96200)

In [5]:
df[(df.year_month >= '2023-09')&(df.feature == fcd.fake_feature)].sum()

shop_id                                          288600
year_month                        2023-092023-102023-11
feature            fake_featurefake_featurefake_feature
feature_value                                       ...
spend_USD                                  37609.374866
impr                                          4966181.0
clicks                                          80866.0
purch                                             425.0
purch_value_USD                            48122.219555
n_ads                                             239.0
dtype: object

In [2]:
df = fcd.save_for_all_shops(db=db, force_from_scratch=True, save_to_s3=True, testing=True)

2023-11-17 11:18:18.680 | INFO     | src.abc.s3_reader_writer:save_for_all_shops:46 - Total of 4633 shops.
2023-11-17 11:18:19.221 | DEBUG    | src.abc.s3_reader_writer:save_for_all_shops:57 - ['dev/descriptive/facebook_creative/main_2023-11-30.csv']
100%|██████████| 1/1 [00:02<00:00,  2.47s/it, shop_id=16038]
2023-11-17 11:18:21.705 | DEBUG    | src.abc.s3_reader_writer:save_for_all_shops:108 - len(main_df) = 0
2023-11-17 11:18:21.705 | DEBUG    | src.abc.s3_reader_writer:save_df:28 - aha: {'df_type': <DescriptiveDF.main: 'main'>, 'df_id': datetime.date(2023, 11, 30)}
2023-11-17 11:18:21.966 | DEBUG    | src.abc.s3_reader_writer:save_df:28 - aha: {'df_type': 'main_failed_shop_ids', 'df_id': datetime.date(2023, 11, 30)}


In [4]:
df

""


In [ ]:
read_df = fcd.read_df(df_type=DescriptiveDF.main, df_id=fcd.end_date)
len(read_df)

2023-11-16 18:02:41.560 | DEBUG    | src.abc.s3_reader_writer:read_df:24 - ####: descriptive/facebook_creative/main_2023-11-30.csv
2023-11-16 18:02:41.561 | DEBUG    | src.utils.s3.read_file_from_s3:read_csv_from_s3:24 - path: dev/descriptive/facebook_creative/main_2023-11-30.csv


1259

In [10]:
read_df.loc[(read_df.feature == fcd.fake_feature)&(read_df.year_month >= '2023-09'),'spend_USD'].sum()

368460.89

In [3]:
fcd.create_and_save_summary()

2023-11-16 18:15:24.112 | DEBUG    | src.abc.s3_reader_writer:read_df:24 - ####: descriptive/facebook_creative/main_2023-11-30.csv
2023-11-16 18:15:24.114 | DEBUG    | src.utils.s3.read_file_from_s3:read_csv_from_s3:24 - path: dev/descriptive/facebook_creative/main_2023-11-30.csv


EmptyDataError: No columns to parse from file

In [4]:
len(df)

1259

In [ ]:
fcd.get_available_dates(df_type=DescriptiveDF.summa)

In [2]:
df = fcd.read_df(df_type = DescriptiveDF.main, df_id=fcd.end_date)

2023-11-16 17:50:24.449 | DEBUG    | src.abc.s3_reader_writer:read_df:24 - ####: descriptive/facebook_creative/main_2023-11-30.csv
2023-11-16 17:50:24.455 | DEBUG    | src.utils.s3.read_file_from_s3:read_csv_from_s3:24 - path: dev/descriptive/facebook_creative/main_2023-11-30.csv


In [6]:
df.loc[(df.feature == fcd.fake_feature)&(df.year_month > '2023-09'),:]

,shop_id,year_month,feature,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads
1214,16038,2023-10,fake_feature,.,152119.2,10156969.0,90161.0,1786.0,292695.91,73.0
1243,16038,2023-11,fake_feature,.,25025.0,1553338.0,15525.0,367.0,53627.43,34.0
2473,16038,2023-10,fake_feature,.,152119.2,10156969.0,90161.0,1786.0,292695.91,73.0
2502,16038,2023-11,fake_feature,.,25025.0,1553338.0,15525.0,367.0,53627.43,34.0
3732,16038,2023-10,fake_feature,.,152119.2,10156969.0,90161.0,1786.0,292695.91,73.0
3761,16038,2023-11,fake_feature,.,25025.0,1553338.0,15525.0,367.0,53627.43,34.0
4991,16038,2023-10,fake_feature,.,152119.2,10156969.0,90161.0,1786.0,292695.91,73.0
5020,16038,2023-11,fake_feature,.,25025.0,1553338.0,15525.0,367.0,53627.43,34.0


In [7]:
shop_df = fcd.get_shop_df(db=db, shop_id=16038)

In [8]:
shop_df.loc[(shop_df.feature == fcd.fake_feature)&(shop_df.year_month > '2023-09'),:]

,shop_id,year_month,feature,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads
1214,16038,2023-10,fake_feature,.,152119.2,10156969.0,90161.0,1786.0,292695.91,73.0
1243,16038,2023-11,fake_feature,.,25025.0,1553338.0,15525.0,367.0,53627.43,34.0


In [2]:
df = fcd.create_and_save_summary()

2023-11-16 17:48:25.333 | DEBUG    | src.abc.s3_reader_writer:read_df:24 - ####: descriptive/facebook_creative/main_2023-11-30.csv
2023-11-16 17:48:25.335 | DEBUG    | src.utils.s3.read_file_from_s3:read_csv_from_s3:24 - path: dev/descriptive/facebook_creative/main_2023-11-30.csv
2023-11-16 17:48:26.696 | DEBUG    | src.abc.s3_reader_writer:save_df:28 - aha: {'df_type': <DescriptiveDF.summary: 'summary'>, 'df_id': datetime.date(2023, 11, 30)}


In [2]:
df = fcd.save_for_all_shops(db=db, df_type=DescriptiveDF.main, testing=True, save_to_s3=True, force_from_scratch=True)

2023-11-16 17:32:29.762 | INFO     | src.abc.s3_reader_writer:save_for_all_shops:43 - Total of 4633 shops.
2023-11-16 17:32:30.423 | DEBUG    | src.abc.s3_reader_writer:save_for_all_shops:51 - ['dev/descriptive/facebook_creative/main_2023-11-30.csv']
100%|██████████| 1/1 [00:03<00:00,  3.64s/it, shop_id=16038]
2023-11-16 17:32:34.078 | DEBUG    | src.abc.s3_reader_writer:save_df:25 - aha: {'df_type': <DescriptiveDF.main: 'main'>, 'df_id': datetime.date(2023, 11, 30)}
2023-11-16 17:32:36.110 | DEBUG    | src.abc.s3_reader_writer:save_df:25 - aha: {'df_type': 'main_failed_shop_ids'}


TypeError: S3ReaderWriter.get_df_path() missing 1 required positional argument: 'df_id'

In [ ]:
fcd.save_df(df=df)

In [3]:
df

,ad_id,account_id,url,shop_id,adset_id,spend,impr,clicks,purch,purch_value,year_month,spend_USD,purch_value_USD
0,23844403447830755,923409341101559,https://scontent-iad3-2.xx.fbcdn.net/v/t45.160...,16038,23844403447940755,21.61,8479.0,36.0,0.0,0.0,2020-02,21.610000,0.0
1,23844403447830755,923409341101559,https://scontent-iad3-2.xx.fbcdn.net/v/t45.160...,16038,23844403447940755,31.94,14783.0,64.0,0.0,0.0,2020-03,31.940000,0.0
2,23844415801920755,923409341101559,https://scontent-iad3-2.xx.fbcdn.net/v/t45.160...,16038,23844415801890755,11.78,3398.0,32.0,0.0,0.0,2020-03,11.780000,0.0
3,23844416005790755,923409341101559,https://scontent-iad3-2.xx.fbcdn.net/v/t45.160...,16038,23844416005780755,6.54,2178.0,14.0,0.0,0.0,2020-03,6.540000,0.0
4,23844416005790755,923409341101559,https://scontent-iad3-2.xx.fbcdn.net/v/t45.160...,16038,23844416005780755,5.24,771.0,11.0,0.0,0.0,2020-08,5.240000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,23858485806510529,1777793985937489,https://scontent-iad3-2.xx.fbcdn.net/v/t45.160...,96200,23858485806520529,6.57,1720.0,20.0,0.0,0.0,2023-09,7.039507,0.0
369,23858485806530529,1777793985937489,https://scontent-iad3-2.xx.fbcdn.net/v/t45.160...,96200,23858485806520529,143.91,26946.0,545.0,0.0,0.0,2023-09,154.194132,0.0
370,23858485806540529,1777793985937489,https://scontent-iad3-2.xx.fbcdn.net/v/t45.160...,96200,23858485806520529,77.74,10711.0,362.0,0.0,0.0,2023-09,83.295475,0.0
371,23861534328770529,1777793985937489,https://scontent-iad3-2.xx.fbcdn.net/v/t45.160...,96200,23853028137570529,7.07,2770.0,92.0,0.0,0.0,2023-10,7.575238,0.0


In [2]:
df = fcd.save_for_all_shops(db=db, df_type=DescriptiveDF.main, testing=True, save_to_s3=False)

2023-11-16 16:40:14.955 | INFO     | src.abc.s3_reader_writer:save_for_all_shops:41 - Total of 4633 shops.
100%|██████████| 5/5 [00:20<00:00,  4.12s/it, shop_id=2]       


In [2]:
df = fcd.create_and_save_main(db=db, testing=True, save_to_s3=False)

2023-11-16 12:45:18.223 | INFO     | src.services.descriptive_saver:create_and_save_main:75 - Total of 4633 nontest shops.
2023-11-16 12:45:51.925 | DEBUG    | src.services.descriptive_saver:create_and_save_main:97 - failed_shop_ids = Series([], Name: shop_id, dtype: object)
2023-11-16 12:45:51.927 | INFO     | src.services.descriptive_saver:create_and_save_main:99 - max_done_shop_id = 161531933
100%|██████████| 5/5 [00:37<00:00,  7.43s/it, shop_id=2]       


In [3]:
df

spend_USD       impr  \
shop_id year_month feature              feature_value                         
2       2019-11    creative_type        carousel        15642.31  1229389.0   
                                        image           92653.45  4215202.0   
                                        video           88720.37  3237266.0   
                   cta                  False          197016.13  8681857.0   
                   discount             False          124359.54  5488967.0   
...                                                          ...        ...   
        2022-11    starts_with_question True              547.01    47829.0   
                   target               acquisition       547.01    47829.0   
                   urgency              False             547.01    47829.0   
                   user_addressing      True              547.01    47829.0   
                   weasel_words         False             547.01    47829.0   

                                                        clicks   purch  \
shop_id year_month feature              feature_value                    
2       2019-11    creative_type        carousel       11557.0   470.0   
                                        image          22405.0  4454.0   
                                        video          26508.0  2402.0   
                   cta                  False          60470.0  7326.0   
                   discount             False          42794.0  4076.0   
...                                                        ...     ...   
        2022-11    starts_with_question True            2565.0     0.0   
                   target               acquisition     2565.0     0.0   
                   urgency              False           2565.0     0.0   
                   user_addressing      True            2565.0     0.0   
                   weasel_words         False           2565.0     0.0   

                                                       purch_value_USD  n_ads  
shop_id year_month feature              feature_value                          
2       2019-11    creative_type        carousel              14093.15    4.0  
                                        image                137420.69   47.0  
                                        video                 60288.67   11.0  
                   cta                  False                211802.51   62.0  
                   discount             False                108425.31   29.0  
...                                                                ...    ...  
        2022-11    starts_with_question True                      0.00    3.0  
                   target               acquisition               0.00    3.0  
                   urgency              False                     0.00    3.0  
                   user_addressing      True                      0.00    3.0  
                   weasel_words         False                     0.00    3.0  

[594868 rows x 6 columns]

In [3]:
df = ftd.read_df(DescriptiveDF.summary)

NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

In [4]:
metrics = (cr, ctr, roas)

In [3]:
df = fcd.create_and_save_summary()

2023-11-14 14:24:20.212 | DEBUG    | src.services.descriptive_saver:create_and_save_summary:24 - 5
2023-11-14 14:24:20.221 | DEBUG    | src.services.descriptive_saver:create_and_save_summary:28 - 5
2023-11-14 14:24:20.227 | DEBUG    | src.services.descriptive_saver:create_and_save_summary:31 - 5
2023-11-14 14:24:20.241 | DEBUG    | src.services.descriptive_saver:create_and_save_summary:35 - 5


In [9]:
df

spend_USD        impr    clicks  \
year_month feature         feature_value                                        
2019-11    creative_type   carousel        26788.812462   2712263.0   21734.0   
                           dynamic           901.356900     78663.0    1313.0   
                           image           92653.450000   4215202.0   22405.0   
                           unknown           429.302778     13906.0     201.0   
                           video           88720.370000   3237266.0   26508.0   
...                                                 ...         ...       ...   
2023-10    urgency         False          171990.024545  12419757.0  123680.0   
           user_addressing False           27375.607347   2312801.0   33868.0   
                           True           144614.417198  10106956.0   89812.0   
           weasel_words    False          171705.122734  12397991.0  123432.0   
                           True              284.901811     21766.0     248.0   

                                           purch  purch_value_USD  n_ads  \
year_month feature         feature_value                                   
2019-11    creative_type   carousel       1132.0     58982.318277   24.0   
                           dynamic           5.0       242.868347   11.0   
                           image          4454.0    137420.690000   47.0   
                           unknown           0.0         0.000000    3.0   
                           video          2402.0     60288.670000   11.0   
...                                          ...              ...    ...   
2023-10    urgency         False          2490.0    350932.531460  226.0   
           user_addressing False           770.0     79675.591463   64.0   
                           True           1720.0    271256.939997  162.0   
           weasel_words    False          2486.0    350412.872271  224.0   
                           True              4.0       519.659189    2.0   

                                          spend_USD_by_shop  impr_by_shop  \
year_month feature         feature_value                                    
2019-11    creative_type   carousel                0.492360      0.555418   
                           dynamic                 0.135998      0.134647   
                           image                   0.156761      0.161839   
                           unknown                 0.064774      0.023803   
                           video                   0.150107      0.124292   
...                                                     ...           ...   
2023-10    urgency         False                   1.000000      1.000000   
           user_addressing False                   0.527904      0.508804   
                           True                    0.472096      0.491196   
           weasel_words    False                   0.994671      0.995535   
                           True                    0.005329      0.004465   

                                          clicks_by_shop  purch_by_shop  \
year_month feature         feature_value                                  
2019-11    creative_type   carousel             0.550264       0.410274   
                           dynamic              0.156198       0.277778   
                           image                0.123505       0.202657   
                           unknown              0.023911       0.000000   
                           video                0.146122       0.109291   
...                                                  ...            ...   
2023-10    urgency         False                1.000000       1.000000   
           user_addressing False                0.552460       0.559171   
                           True                 0.447540       0.440829   
           weasel_words    False                0.996835       0.994645   
                           True                 0.003165       0.005355   

                         

In [5]:
metrics = (cr, ctr, roas)
main_df = fcd.read_df(df_type=DescriptiveDF.main)
for metric in metrics:
    main_df[str(metric)] = main_df.apply(metric.formula_series, axis=1)

In [6]:
main_df.shop_id.unique()

array([   16038, 44301396, 34810574,    96200,        2])

In [12]:
main_df[main_df.ROAS > 30000]

,year_month,feature,feature_value,shop_id,spend_USD,impr,clicks,purch,purch_value_USD,n_ads,n_shops,spend_USD_by_shop,impr_by_shop,clicks_by_shop,purch_by_shop,purch_value_USD_by_shop,n_ads_by_shop,CR,CTR,ROAS
1637,2023-08,creative_type,carousel,44317434,4.607232,1054.0,40.0,17.0,1997.584494,3.0,2,0.000502,0.000477,0.002351,0.005768,0.006215,0.020455,42.5,3.795066,43357.587368


In [4]:
main_df[main_df.feature == 'discount_list']

,shop_id,year_month,feature,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads
6,16038,2020-02,discount_list,70,617.57,167202.0,1001.0,2.0,155.00,117.0
32,16038,2020-03,discount_list,False,134.44,52842.0,454.0,2.0,296.90,10.0
33,16038,2020-03,discount_list,10,53.22,10904.0,49.0,2.0,367.05,3.0
34,16038,2020-03,discount_list,70,1479.84,448708.0,4154.0,22.0,3052.15,49.0
60,16038,2020-04,discount_list,False,1668.38,334815.0,3568.0,129.0,7751.90,4.0
...,...,...,...,...,...,...,...,...,...,...
4412,2,2021-08,discount_list,15,10156.95,669965.0,1287.0,91.0,3567.30,35.0
4413,2,2021-08,discount_list,25,26241.43,1468132.0,3063.0,345.0,14107.18,51.0
4440,2,2022-06,discount_list,False,626.67,27796.0,281.0,0.0,0.00,8.0
4458,2,2022-07,discount_list,False,1182.74,102443.0,251.0,0.0,0.00,5.0


In [20]:
summary_df = fcd.read_df(df_type=DescriptiveDF.summary)

In [24]:
summary_df.groupby(['year_month', 'feature', 'feature_value']).sum()

spend_USD        impr    clicks  \
year_month feature         feature_value                                        
2019-11    creative_type   carousel        26788.812462   2712263.0   21734.0   
                           dynamic           901.356900     78663.0    1313.0   
                           image           92653.450000   4215202.0   22405.0   
                           unknown           429.302778     13906.0     201.0   
                           video           88720.370000   3237266.0   26508.0   
...                                                 ...         ...       ...   
2023-10    urgency         False          171990.024545  12419757.0  123680.0   
           user_addressing False           27375.607347   2312801.0   33868.0   
                           True           144614.417198  10106956.0   89812.0   
           weasel_words    False          171705.122734  12397991.0  123432.0   
                           True              284.901811     21766.0     248.0   

                                           purch  purch_value_USD  n_ads  \
year_month feature         feature_value                                   
2019-11    creative_type   carousel       1132.0     58982.318277   24.0   
                           dynamic           5.0       242.868347   11.0   
                           image          4454.0    137420.690000   47.0   
                           unknown           0.0         0.000000    3.0   
                           video          2402.0     60288.670000   11.0   
...                                          ...              ...    ...   
2023-10    urgency         False          2490.0    350932.531460  226.0   
           user_addressing False           770.0     79675.591463   64.0   
                           True           1720.0    271256.939997  162.0   
           weasel_words    False          2486.0    350412.872271  224.0   
                           True              4.0       519.659189    2.0   

                                          spend_USD_by_shop  impr_by_shop  \
year_month feature         feature_value                                    
2019-11    creative_type   carousel                0.492360      0.555418   
                           dynamic                 0.135998      0.134647   
                           image                   0.156761      0.161839   
                           unknown                 0.064774      0.023803   
                           video                   0.150107      0.124292   
...                                                     ...           ...   
2023-10    urgency         False                   1.000000      1.000000   
           user_addressing False                   0.527904      0.508804   
                           True                    0.472096      0.491196   
           weasel_words    False                   0.994671      0.995535   
                           True                    0.005329      0.004465   

                                          clicks_by_shop  purch_by_shop  \
year_month feature         feature_value                                  
2019-11    creative_type   carousel             0.550264       0.410274   
                           dynamic              0.156198       0.277778   
                           image                0.123505       0.202657   
                           unknown              0.023911       0.000000   
                           video                0.146122       0.109291   
...                                                  ...            ...   
2023-10    urgency         False                1.000000       1.000000   
           user_addressing False                0.552460       0.559171   
                           True                 0.447540       0.440829   
           weasel_words    False                0.996835       0.994645   
                           True                 0.003165       0.005355   

                         

In [25]:
main_df = fcd.read_df(df_type=DescriptiveDF.main)

sum_by_shop_and_feature = main_df.groupby(["shop_id", "year_month", "feature"])[
    fcd.metric_columns
].sum()
grouped_by_shop_and_value = main_df.groupby(["shop_id", "year_month", "feature", "feature_value"])

absolute_sum_by_shop_and_value = grouped_by_shop_and_value[fcd.metric_columns].sum()
relative_sum_by_shop_and_value = (
    absolute_sum_by_shop_and_value[fcd.metric_columns] / sum_by_shop_and_feature
)
relative_sum_by_value = (
    relative_sum_by_shop_and_value.reset_index()
    .groupby(["year_month", "feature", "feature_value"])[fcd.metric_columns]
    .sum()
)
n_shops_by_month_and_feature = (
    relative_sum_by_value.reset_index().groupby(["year_month", "feature"])[fcd.metric_columns].sum()
)
normalised_relative_sum_by_value = relative_sum_by_value.div(n_shops_by_month_and_feature, axis=0)
normalised_relative_sum_by_value.rename(
    columns={col: col + "_by_shop" for col in fcd.metric_columns}, inplace=True
)
absolute_sum_by_value = (
    absolute_sum_by_shop_and_value.reset_index()
    .groupby(["year_month", "feature", "feature_value"])
    .sum()
).drop(columns=['shop_id'])
summary_df = absolute_sum_by_value.join(normalised_relative_sum_by_value)

In [26]:
absolute_sum_by_shop_and_value

spend_USD       impr  \
shop_id  year_month feature         feature_value                             
2        2019-11    creative_type   carousel        15642.310000  1229389.0   
                                    image           92653.450000  4215202.0   
                                    video           88720.370000  3237266.0   
                    cta             False          197016.130000  8681857.0   
                    discount        False          124359.540000  5488967.0   
...                                                          ...        ...   
44301396 2023-10    target          remarketing       455.270908   106266.0   
                    urgency         False            2051.496652   637797.0   
                    user_addressing False            2050.137262   637483.0   
                                    True                1.359391      314.0   
                    weasel_words    False            2051.496652   637797.0   

                                                    clicks   purch  \
shop_id  year_month feature         feature_value                    
2        2019-11    creative_type   carousel       11557.0   470.0   
                                    image          22405.0  4454.0   
                                    video          26508.0  2402.0   
                    cta             False          60470.0  7326.0   
                    discount        False          42794.0  4076.0   
...                                                    ...     ...   
44301396 2023-10    target          remarketing     1053.0   128.0   
                    urgency         False           7399.0   455.0   
                    user_addressing False           7396.0   455.0   
                                    True               3.0     0.0   
                    weasel_words    False           7399.0   455.0   

                                                   purch_value_USD  n_ads  
shop_id  year_month feature         feature_value                          
2        2019-11    creative_type   carousel          14093.150000    4.0  
                                    image            137420.690000   47.0  
                                    video             60288.670000   11.0  
                    cta             False            211802.510000   62.0  
                    discount        False            108425.310000   29.0  
...                                                            ...    ...  
44301396 2023-10    target          remarketing        7702.049520    7.0  
                    urgency         False             27865.477653   28.0  
                    user_addressing False             27865.477653   25.0  
                                    True                  0.000000    3.0  
                    weasel_words    False             27865.477653   28.0  

[4491 rows x 6 columns]

In [29]:
main_df.set_index(["shop_id", "year_month", "feature", "feature_value"]) - absolute_sum_by_shop_and_value

/tmp/ipykernel_61020/2655004074.py:1: RuntimeWarning: The values in the array are unorderable. Pass `sort=False` to suppress this warning.
  main_df.set_index(["shop_id", "year_month", "feature", "feature_value"]) - absolute_sum_by_shop_and_value


spend_USD  impr  \
shop_id year_month feature              feature_value                    
16038   2020-02    creative_type        carousel             0.0   0.0   
                                        dynamic              0.0   0.0   
                                        image                0.0   0.0   
                                        video                0.0   0.0   
                   cta                  False                0.0   0.0   
...                                                          ...   ...   
2       2022-11    starts_with_question True                 0.0   0.0   
                   target               acquisition          0.0   0.0   
                   urgency              False                0.0   0.0   
                   user_addressing      True                 0.0   0.0   
                   weasel_words         False                0.0   0.0   

                                                       clicks  purch  \
shop_id year_month feature              feature_value                  
16038   2020-02    creative_type        carousel          0.0    0.0   
                                        dynamic           0.0    0.0   
                                        image             0.0    0.0   
                                        video             0.0    0.0   
                   cta                  False             0.0    0.0   
...                                                       ...    ...   
2       2022-11    starts_with_question True              0.0    0.0   
                   target               acquisition       0.0    0.0   
                   urgency              False             0.0    0.0   
                   user_addressing      True              0.0    0.0   
                   weasel_words         False             0.0    0.0   

                                                       purch_value_USD  n_ads  
shop_id year_month feature              feature_value                          
16038   2020-02    creative_type        carousel                   0.0    0.0  
                                        dynamic                    0.0    0.0  
                                        image                      0.0    0.0  
                                        video                      0.0    0.0  
                   cta                  False                      0.0    0.0  
...                                                                ...    ...  
2       2022-11    starts_with_question True                       0.0    0.0  
                   target               acquisition                0.0    0.0  
                   urgency              False                      0.0    0.0  
                   user_addressing      True                       0.0    0.0  
                   weasel_words         False                      0.0    0.0  

[4491 rows x 6 columns]

In [16]:
relative_sum_by_value.reset_index().groupby(['year_month','feature']).sum()

feature_value  spend_USD  \
year_month feature                                                             
2019-11    creative_type         carouseldynamicimageunknownvideo        3.0   
           cta                                              False        3.0   
           discount                                             1        3.0   
           discount_list                                       40        3.0   
           emoji                                                1        3.0   
...                                                           ...        ...   
2023-10    starts_with_question                                 1        3.0   
           target                          acquisitionremarketing        3.0   
           urgency                                          False        3.0   
           user_addressing                                      1        3.0   
           weasel_words                                         1        3.0   

                                 impr  clicks  purch  purch_value_USD  n_ads  
year_month feature                                                            
2019-11    creative_type          3.0     3.0    3.0              3.0    3.0  
           cta                    3.0     3.0    3.0              3.0    3.0  
           discount               3.0     3.0    3.0              3.0    3.0  
           discount_list          3.0     3.0    3.0              3.0    3.0  
           emoji                  3.0     3.0    3.0              3.0    3.0  
...                               ...     ...    ...              ...    ...  
2023-10    starts_with_question   3.0     3.0    3.0              3.0    3.0  
           target                 3.0     3.0    3.0              3.0    3.0  
           urgency                3.0     3.0    3.0              3.0    3.0  
           user_addressing        3.0     3.0    3.0              3.0    3.0  
           weasel_words           3.0     3.0    3.0              3.0    3.0  

[816 rows x 7 columns]

In [6]:
read_df = fcd.read_df(DescriptiveDF.summary)

In [9]:
read_df[read_df.feature == 'discount_list']

,year_month,feature,feature_value,shop_id,spend_USD,impr,clicks,purch,purch_value_USD,n_ads,n_shops,spend_USD_by_shop,impr_by_shop,clicks_by_shop,purch_by_shop,purch_value_USD_by_shop,n_ads_by_shop
8,2019-11,discount_list,False,44397598,136836.702140,7.064410e+06,54485.000000,4743.000000,153557.346624,63.000000,3,0.877072,0.877411,0.902563,0.852125,0.837306,0.822581
9,2019-11,discount_list,40,2,72656.590000,3.192890e+06,17676.000000,3250.000000,103377.200000,33.000000,1,0.122928,0.122589,0.097437,0.147875,0.162694,0.177419
38,2019-12,discount_list,False,44397598,132768.681712,9.141173e+06,71607.000000,4689.000000,139540.361599,56.000000,3,0.949539,0.968503,0.980037,0.933978,0.929269,0.883598
39,2019-12,discount_list,40,2,22115.490000,8.270190e+05,4150.000000,1085.000000,32017.430000,22.000000,1,0.050461,0.031497,0.019963,0.066022,0.070731,0.116402
66,2020-01,discount_list,False,44301398,240576.640470,1.894438e+07,146599.000000,8912.000000,258811.204017,63.000000,2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1724,2023-10,discount_list,35,96200,5461.200923,5.072025e+05,5765.066667,77.933333,9879.664075,20.866667,1,0.102159,0.104042,0.073572,0.104328,0.108433,0.055644
1725,2023-10,discount_list,60,96200,1592.772916,1.494381e+05,1634.133333,20.366667,2377.553295,9.233333,1,0.029795,0.030654,0.020854,0.027265,0.026094,0.024622
1726,2023-10,discount_list,80,16038,84238.740000,5.236637e+06,43537.000000,731.000000,119135.940000,29.000000,1,0.184589,0.171857,0.160960,0.136432,0.135677,0.132420
1727,2023-10,discount_list,90,96200,8039.864824,6.494574e+05,12076.800000,144.700000,17374.081757,14.900000,1,0.150396,0.133223,0.154119,0.193708,0.190686,0.039733


In [5]:
df = fcd.create_and_save_main(db=db, save_to_s3=False, testing=True, force_from_scratch=True)

2023-11-14 12:37:10.326 | INFO     | src.services.descriptive_saver:create_and_save_main:65 - Total of 4633 nontest shops.
100%|██████████| 1/1 [00:02<00:00,  2.69s/it, shop_id=16038]


In [20]:
df = fcd.get_shop_descriptive_df(db=db, shop_id=16038)

In [16]:
shop_df = ping_facebook_creative_and_performance(db=db, shop_id = 16038)
shop_df['n_ads'] = 1
shop_df.spend_USD.sum()

4027304.4400000004

In [17]:
for explode_col in fcd.explode_descriptive_columns:
    shop_df[fcd.metric_columns] = shop_df[fcd.metric_columns].div(
        shop_df[explode_col].apply(len), axis=0
    )
    shop_df = shop_df.explode(explode_col).dropna(axis=0, subset=explode_col)

In [6]:
df.loc[df.index.get_level_values('feature') == 'discount_list', 'spend_USD'].sum()

3911730.81

In [7]:
df.loc[df.index.get_level_values('feature') == 'emoji', 'spend_USD'].sum()

3911730.8100000005

In [7]:
df.loc[df.index.get_level_values('feature') == 'discount_list', :]

spend_USD       impr   clicks  \
shop_id year_month feature       feature_value                                  
16038   2020-02    discount_list 70                617.57   167202.0   1001.0   
        2020-03    discount_list False             134.44    52842.0    454.0   
                                 10                 53.22    10904.0     49.0   
                                 70               1479.84   448708.0   4154.0   
        2020-04    discount_list False            1668.38   334815.0   3568.0   
...                                                   ...        ...      ...   
        2023-08    discount_list 80              72775.74  3575541.0  46167.0   
        2023-09    discount_list False          113985.59  7952817.0  75382.0   
                                 80              77331.10  3724984.0  43874.0   
        2023-10    discount_list False           67880.46  4920332.0  46624.0   
                                 80              84238.74  5236637.0  43537.0   

                                                 purch  purch_value_USD  n_ads  
shop_id year_month feature       feature_value                                  
16038   2020-02    discount_list 70                2.0           155.00  117.0  
        2020-03    discount_list False             2.0           296.90   10.0  
                                 10                2.0           367.05    3.0  
                                 70               22.0          3052.15   49.0  
        2020-04    discount_list False           129.0          7751.90    4.0  
...                                                ...              ...    ...  
        2023-08    discount_list 80              612.0        108561.79    7.0  
        2023-09    discount_list False          1779.0        292788.50   52.0  
                                 80              684.0        120949.83   20.0  
        2023-10    discount_list False          1055.0        173559.97   44.0  
                                 80              731.0        119135.94   29.0  

[104 rows x 6 columns]

In [4]:
df.loc[df.index.get_level_values('feature') == 'fake_feature', 'spend_USD'].sum()

103139182.53525747

In [7]:
df.loc[df.index.get_level_values('feature') == 'discount_list', 'spend_USD'].reset_index()

,shop_id,year_month,feature,feature_value,spend_USD
0,16038,2020-02,discount_list,70,617.570
1,16038,2020-03,discount_list,10,53.220
2,16038,2020-03,discount_list,70,1479.840
3,16038,2020-04,discount_list,10,108.460
4,16038,2020-04,discount_list,70,350.180
5,16038,2020-05,discount_list,70,138.770
6,16038,2020-06,discount_list,70,147.650
7,16038,2020-07,discount_list,10,3.900
8,16038,2020-07,discount_list,70,452.260
9,16038,2020-08,discount_list,10,0.020


In [5]:
df.loc[df.index.get_level_values('feature') == 'emoji', 'spend_USD'].sum()

2167861.83

In [2]:
df = fcd.get_shop_descriptive_df(db=db, shop_id=16038)

In [2]:
creative_df = fcd.read_df(df_type=DescriptiveDF.main, end_date = date())
target_df = ftd.read_df(df_type=DescriptiveDF.main)
creative_fake = creative_df[creative_df.feature=='fake_feature']
target_fake = target_df[target_df.feature=='fake_feature']

In [3]:
creative_df = fcd.read_df(df_type=DescriptiveDF.main)

In [5]:
creative_df

array(['creative_type', 'cta', 'discount', 'discount_list', 'emoji',
       'fact_words', 'fake_feature', 'free_shipping', 'hashtag', 'link',
       'percentage', 'price', 'starts_with_question', 'target', 'urgency',
       'user_addressing', 'weasel_words'], dtype=object)

In [2]:
old_df = fcd.read_df(df_type=DescriptiveDF.main, end_date = date(year=2023, month=9, day=30))
new_df = fcd.read_df(df_type=DescriptiveDF.main)

In [4]:
old_sep = old_df[(old_df.year_month == '2023-09') & (old_df.feature == 'fake_feature')]
new_sep = new_df[(new_df.year_month == '2023-09') & (new_df.feature == 'fake_feature')]

In [7]:
159350 in new_sep.shop_id.tolist()

True

In [20]:
sorted(set(old_sep.shop_id.unique()) - set(new_sep.shop_id.unique()))

[159350,
 199383,
 231517,
 268714,
 335856,
 1259996,
 1912299,
 2058949,
 2387181,
 2410663,
 2423816,
 2577155,
 3397559,
 7629029,
 7986318,
 8588771,
 8966477,
 9180676,
 9662886,
 9927170,
 10248339,
 10260559,
 10543255,
 11883461,
 12109463,
 12195110,
 12348254,
 12451478,
 12480682,
 12760428,
 12787254,
 13512604,
 14018822,
 15433380,
 15666148,
 16200158,
 16621201,
 17426875,
 18166172,
 18675218,
 18686172,
 19505868,
 19741878,
 20134299,
 20329307,
 20500879,
 20980695,
 21575323,
 22119365,
 22502944,
 22660537,
 22926764,
 23224328,
 23647289,
 24268796,
 24640760,
 25254472,
 25963515,
 26389028,
 28794880,
 29080200,
 29413487,
 29633032,
 29842229,
 29851960,
 29932371,
 30929283,
 31342479,
 31420164,
 31522526,
 31547008,
 31548005,
 31584044,
 31587104,
 31728156,
 31839270,
 31859716,
 32795319,
 33089105,
 33166821,
 33212206,
 33363962,
 34237718,
 34248783,
 34366561,
 35522045,
 35756063,
 36183854,
 36233823,
 37111565,
 37128351,
 37211363,
 37254609,
 3

In [3]:
aha = fcd.get_shop_descriptive_df(db=db, shop_id=159350).reset_index()

In [4]:
aha[aha.year_month == '2023-09']

,shop_id,year_month,feature,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads
1201,159350,2023-09,creative_type,carousel,152.649306,14972.0,27.0,9.0,588.765347,2
1202,159350,2023-09,creative_type,dynamic,976.726667,53117.0,222.0,21.0,1326.507551,15
1203,159350,2023-09,creative_type,image,23658.295607,1692034.0,3081.0,645.0,36824.359857,82
1204,159350,2023-09,creative_type,unknown,275664.242512,17440544.0,68462.0,5720.0,322389.417261,815
1205,159350,2023-09,creative_type,video,37501.560972,2433908.0,10148.0,827.0,43473.057839,105
1206,159350,2023-09,cta,False,337260.019557,21564561.0,81826.0,7170.0,401454.289957,1015
1207,159350,2023-09,cta,True,693.455507,70014.0,114.0,52.0,3147.817899,4
1208,159350,2023-09,discount,False,314295.179457,19942541.0,78859.0,6577.0,367777.747999,937
1209,159350,2023-09,discount,True,23658.295607,1692034.0,3081.0,645.0,36824.359857,82
1210,159350,2023-09,emoji,False,205293.165767,12430400.0,53162.0,3956.0,217571.562059,471


In [15]:
new_sep.shop_id.nunique()

279

In [26]:
creative_fake = creative_df[creative_df.feature=='fake_feature']
target_fake = target_df[target_df.feature=='fake_feature']
# creative_fake.year_month = creative_fake.year_month.apply(lambda year_month: datetime.strptime(year_month, '%Y-%m'))
# target_fake.year_month = target_fake.year_month.apply(lambda year_month: datetime.strptime(year_month, '%Y-%m'))
creative_filtered = creative_fake[creative_fake.year_month == '2023-09']
target_filtered = target_fake[target_fake.year_month == '2023-09']

In [28]:
creative_filtered

,shop_id,year_month,feature,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads
694,16038,2023-09,fake_feature,.,191316.690000,11677801.0,119256.0,2463.0,413738.330000,72
1377,96200,2023-09,fake_feature,.,17409.352019,3111781.0,50316.0,164.0,16121.205929,102
2223,159350,2023-09,fake_feature,.,341576.483037,21536511.0,81642.0,7208.0,409750.964442,993
2754,199383,2023-09,fake_feature,.,6806.650000,196650.0,1469.0,114.0,8568.090000,44
3080,231517,2023-09,fake_feature,.,246.570000,5406.0,157.0,3.0,97.970000,5
...,...,...,...,...,...,...,...,...,...,...
302882,147893909,2023-09,fake_feature,.,3946.674603,185369.0,3936.0,110.0,13219.804687,166
303018,148020088,2023-09,fake_feature,.,18.330000,3179.0,163.0,0.0,0.000000,1
303688,148136527,2023-09,fake_feature,.,3646.294400,455595.0,3054.0,164.0,27601.193838,74
304340,148231640,2023-09,fake_feature,.,10604.099153,3142043.0,44080.0,2057.0,82398.739107,105


In [39]:
weird_shop_ids = [shop_id for shop_id in target_filtered.shop_id.tolist() if shop_id not in creative_filtered.shop_id.tolist()]
print(f'{len(weird_shop_ids)=}')
normal_bastards = [shop_id for shop_id in creative_filtered.shop_id.tolist() if shop_id not in target_filtered.shop_id.tolist()]

len(weird_shop_ids)=1


In [40]:
normal_bastards

[96200,
 159350,
 199383,
 231517,
 268714,
 335856,
 1259996,
 1912299,
 2058949,
 2387181,
 2410663,
 2423816,
 2577155,
 3397559,
 7629029,
 7986318,
 8588771,
 8966477,
 9180676,
 9662886,
 9927170,
 10248339,
 10260559,
 10543255,
 11883461,
 12109463,
 12195110,
 12348254,
 12451478,
 12480682,
 12760428,
 12787254,
 13512604,
 14018822,
 15433380,
 15666148,
 16200158,
 16621201,
 17426875,
 18166172,
 18675218,
 18686172,
 19505868,
 19741878,
 20134299,
 20329307,
 20500879,
 20980695,
 21575323,
 22119365,
 22502944,
 22660537,
 22926764,
 23224328,
 23647289,
 24268796,
 24640760,
 25254472,
 25963515,
 26389028,
 28794880,
 29080200,
 29413487,
 29633032,
 29842229,
 29851960,
 29932371,
 30929283,
 31342479,
 31420164,
 31522526,
 31547008,
 31548005,
 31584044,
 31587104,
 31728156,
 31839270,
 31859716,
 32795319,
 33089105,
 33166821,
 33212206,
 33363962,
 34237718,
 34248783,
 34366561,
 35522045,
 35756063,
 36183854,
 36233823,
 37111565,
 37128351,
 37211363,
 3725

In [43]:
hmm = ftd.get_shop_descriptive_df(db=db, shop_id=96200, start_date=ftd.start_date, end_date=ftd.end_date).reset_index()

In [2]:
target_new = ftd.create_and_save_main(db=db, force_from_scratch=True, testing=True, save_to_s3=False)

100%|██████████| 4/4 [00:23<00:00,  5.99s/it, shop_id=96200, len=242]   


In [10]:
target_new.reset_index().shop_id.unique().tolist()

[16038, 44301396, 34810574, 96200]

In [4]:
target_done_shop_ids = ftd.read_df(df_type=DescriptiveDF.done_shop_ids)

In [6]:
creative_done_shop_ids = fcd.read_df(df_type=DescriptiveDF.done_shop_ids)

In [7]:
creative_done_shop_ids

,shop_id
0,38439301
1,38439792
2,38439899
3,38440624
4,38440735
5,38440851
6,38441170
7,38441282
8,38441552
9,38441675


In [3]:
target_new

spend_USD       impr  \
shop_id year_month feature      feature_value                            
16038   2021-10    age_groups   middle          4801.943333   219289.0   
                                old             4801.943333   219289.0   
                                young           4801.943333   219289.0   
                   audience     broad          12591.340000   601249.0   
                                retargeting     1814.490000    56618.0   
...                                                     ...        ...   
96200   2023-09    audience     retargeting      597.941923    77847.0   
                   fake_feature .              17312.110183  3111781.0   
                   gender       both           17312.110183  3111781.0   
                   target       acquisition    16714.168260  3033934.0   
                                remarketing      597.941923    77847.0   

                                                     clicks  purch  \
shop_id year_month feature      feature_value                        
16038   2021-10    age_groups   middle          2814.333333   55.0   
                                old             2814.333333   55.0   
                                young           2814.333333   55.0   
                   audience     broad           7403.000000   91.0   
                                retargeting     1040.000000   74.0   
...                                                     ...    ...   
96200   2023-09    audience     retargeting      242.000000   14.0   
                   fake_feature .              50316.000000  164.0   
                   gender       both           50316.000000  164.0   
                   target       acquisition    50074.000000  150.0   
                                remarketing      242.000000   14.0   

                                               purch_value_USD       n_ads  
shop_id year_month feature      feature_value                               
16038   2021-10    age_groups   middle            10971.360000    4.666667  
                                old               10971.360000    4.666667  
                                young             10971.360000    4.666667  
                   audience     broad             18269.520000    8.000000  
                                retargeting       14644.560000    6.000000  
...                                                        ...         ...  
96200   2023-09    audience     retargeting        1905.307533   14.000000  
                   fake_feature .                 16031.159174  102.000000  
                   gender       both              16031.159174  102.000000  
                   target       acquisition       14125.851640   88.000000  
                                remarketing        1905.307533   14.000000  

[757 rows x 6 columns]

In [44]:
hmm[hmm.year_month == '2023-09']

,shop_id,year_month,feature,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads
231,96200,2023-09,age_groups,middle,5770.703394,1.037260e+06,16772.0,54.666667,5343.719725,34.0
232,96200,2023-09,age_groups,old,5770.703394,1.037260e+06,16772.0,54.666667,5343.719725,34.0
233,96200,2023-09,age_groups,young,5770.703394,1.037260e+06,16772.0,54.666667,5343.719725,34.0
234,96200,2023-09,audience,broad,1471.142416,2.337810e+05,4689.0,12.000000,1216.516980,16.0
235,96200,2023-09,audience,interest,14820.151345,2.721560e+06,40458.0,138.000000,12909.334660,58.0
236,96200,2023-09,audience,lookalike,422.874499,7.859300e+04,4927.0,0.000000,0.000000,14.0
237,96200,2023-09,audience,retargeting,597.941923,7.784700e+04,242.0,14.000000,1905.307533,14.0
238,96200,2023-09,fake_feature,.,17312.110183,3.111781e+06,50316.0,164.000000,16031.159174,102.0
239,96200,2023-09,gender,both,17312.110183,3.111781e+06,50316.0,164.000000,16031.159174,102.0
240,96200,2023-09,target,acquisition,16714.168260,3.033934e+06,50074.0,150.000000,14125.851640,88.0


In [45]:
creative_filtered[creative_filtered.shop_id == 96200]

,shop_id,year_month,feature,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads
1377,96200,2023-09,fake_feature,.,17409.352019,3111781.0,50316.0,164.0,16121.205929,102


In [31]:
target_filtered.groupby('shop_id').apply(lambda df: len(df))

shop_id
16038        1
44301396     1
44542158     1
45030209     1
45030541     1
            ..
147893909    1
148020088    1
148136527    1
148231640    1
148705867    1
Length: 249, dtype: int64

In [ ]:
creative_fake.groupby('shop_id')

In [16]:
creative_row = creative_df[(creative_df.shop_id == 16038) & (creative_df.year_month == '2022-09') & (creative_df.feature == 'fake_feature')]
target_row = target_df[(target_df.shop_id == 16038) & (target_df.year_month == '2022-09') & (target_df.feature == 'fake_feature')]
creative_row

,shop_id,year_month,feature,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads
394,16038,2022-09,fake_feature,.,225523.36,8979363.0,84937.0,1645.0,301204.49,20


In [17]:
target_row

,shop_id,year_month,feature,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads
112,16038,2022-09,fake_feature,.,225523.36,8979363.0,84937.0,1645.0,301204.49,20.0


In [14]:
target_row = target_df[(target_df.shop_id == 16038) & (target_df.year_month == '2022-09') & (target_df.feature == 'fake_feature')]

In [10]:
(creative_df.spend_USD.sum() - target_df.spend_USD.sum()) / creative_df.spend_USD.sum()

0.86637385262137

In [9]:
target_df.spend_USD.sum()

357250009.40284485

In [8]:
sorted(df.year_month.unique().tolist())

['2021-09',
 '2021-10',
 '2021-11',
 '2021-12',
 '2022-01',
 '2022-02',
 '2022-03',
 '2022-04',
 '2022-05',
 '2022-06',
 '2022-07',
 '2022-08',
 '2022-09',
 '2022-10',
 '2022-11',
 '2022-12',
 '2023-01',
 '2023-02',
 '2023-03',
 '2023-04',
 '2023-05',
 '2023-06',
 '2023-07',
 '2023-08']

In [2]:
df = fcd.get_shop_descriptive_df(db=db, shop_id=16038, start_date=fcd.start_date, end_date=fcd.end_date)

2023-08-23 12:06:28.465 | DEBUG    | src.abc.descriptive:get_shop_descriptive_df:79 - 16038: +


In [4]:
df.reset_index()

,year_month,feature,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads
0,2021-08,creative_type,carousel,39472.02,2356863.0,30647.0,417.0,81866.18,10
1,2021-08,creative_type,dynamic,7330.64,824425.0,5533.0,50.0,12367.94,1
2,2021-08,cta,False,46802.66,3181288.0,36180.0,467.0,94234.12,11
3,2021-08,discount,False,32465.40,2318380.0,26304.0,343.0,69488.28,9
4,2021-08,discount,True,14337.26,862908.0,9876.0,124.0,24745.84,2
...,...,...,...,...,...,...,...,...,...
593,2023-07,target,remarketing,9733.30,358712.0,2686.0,588.0,93628.17,9
594,2023-07,urgency,False,247202.34,15707007.0,159885.0,2384.0,427760.61,60
595,2023-07,user_addressing,False,9096.61,598525.0,4972.0,42.0,7534.20,3
596,2023-07,user_addressing,True,238105.73,15108482.0,154913.0,2342.0,420226.41,57


In [4]:
fcd.end_date

datetime.date(2023, 7, 31)

In [ ]:
df = fcd.create_and_save_main(db=db, end_date=end_date, testing=True)

2023-08-23 07:23:25.218 | DEBUG    | src.services.descriptive.descriptive_saver:create_and_save_main:72 - list_of_objects_on_s3 = []
  0%|          | 0/2 [00:00<?, ?it/s]2023-08-23 07:23:29.479 | DEBUG    | src.abc.descriptive.descriptive:get_shop_descriptive_df:26 - 16038: +
2023-08-23 07:23:29.481 | ERROR    | src.services.descriptive.descriptive_saver:create_and_save_main:132 - Error for shop id 16038: 
 local variable 'df' referenced before assignment
 50%|█████     | 1/2 [00:04<00:04,  4.25s/it]2023-08-23 07:23:38.422 | DEBUG    | src.abc.descriptive.descriptive:get_shop_descriptive_df:26 - 44301396: +
2023-08-23 07:23:38.424 | ERROR    | src.services.descriptive.descriptive_saver:create_and_save_main:132 - Error for shop id 44301396: 
 local variable 'df' referenced before assignment
100%|██████████| 2/2 [00:13<00:00,  6.60s/it]


In [2]:
df = fcd.get_shop_descriptive_df(db=db, shop_id=16038, start_date=start_date, end_date=end_date)

2023-08-23 07:27:32.571 | DEBUG    | src.abc.descriptive.descriptive:get_shop_descriptive_df:26 - 16038: +


In [5]:
df[df.index.get_level_values('feature').isin([fcd.fake_feature])]

,,,spend_USD,impr,clicks,purch,purch_value_USD,n_ads
year_month,feature,feature_value,,,,,,
2021-08,fake_feature,.,46802.66,3181288.0,36180.0,467.0,94234.12,11
2021-09,fake_feature,.,14685.42,644275.0,8119.0,182.0,33582.36,10
2021-10,fake_feature,.,14405.83,657867.0,8443.0,165.0,32914.08,14
2021-11,fake_feature,.,29712.06,1085548.0,12703.0,283.0,65533.72,16
2021-12,fake_feature,.,37576.15,2064083.0,25338.0,230.0,55109.50,28
2022-01,fake_feature,.,128957.96,7752832.0,88656.0,915.0,207634.21,28
2022-02,fake_feature,.,190522.86,8395216.0,85671.0,1228.0,265496.12,19
2022-03,fake_feature,.,239163.02,9819873.0,97975.0,1717.0,372800.38,24
2022-04,fake_feature,.,82381.74,2804193.0,34621.0,858.0,155961.30,18


In [2]:
main_df = fcd.read_df('main', end_date=end_date)

In [3]:
main_df.apply(cr.formula_series, axis=1)

0         7.784149
1         0.000000
2         0.000000
3         0.000000
4         8.314337
            ...   
298913    0.609794
298914    0.384568
298915    0.664939
298916    0.556147
298917    0.475170
Length: 298918, dtype: float64

In [4]:
df = pd.DataFrame()

In [5]:
df[cr] = [5]

In [7]:
df.loc[0, cr]

5

In [8]:
cr

In [17]:
main_df.purch_value_USD.quantile(q=0.35)

64.55234353073253

In [18]:
conversion_df = main_df[main_df.purch_value_USD > 0]

In [20]:
conversion_df.clicks.describe()

count    2.048710e+05
mean     2.017217e+04
std      1.108909e+05
min      0.000000e+00
25%      7.220000e+02
50%      3.058000e+03
75%      1.141450e+04
max      4.408504e+06
Name: clicks, dtype: float64

In [101]:
normalised_relative_sum_by_value = relative_sum_by_value.div(n_shops_by_month_and_feature, axis=0)

In [104]:
absolute_sum_by_value = absolute_sum_by_shop_and_value.reset_index().groupby(['year_month', 'feature', 'feature_value'])[fcd.metric_columns].sum()

In [106]:
normalised_relative_sum_by_value.reset_index().groupby(['year_month', 'feature'])[fcd.metric_columns].sum()
normalised_relative_sum_by_value.rename(columns = {col: col+'_by_shop' for col in fcd.metric_columns}, inplace=True)

In [107]:
summary_df = absolute_sum_by_value.join(normalised_relative_sum_by_value)

In [108]:
summary_df

spend_USD         impr  \
year_month feature         feature_value                              
2021-08    creative_type   carousel       1.385065e+06  189680897.0   
                           dynamic        2.015461e+06  231883453.0   
                           image          1.127209e+06  179940512.0   
                           unknown        1.303205e+06  193829477.0   
                           video          1.305978e+06   95865870.0   
...                                                ...          ...   
2023-07    urgency         True           6.186890e+05  110600249.0   
           user_addressing False          1.933799e+06  372190025.0   
                           True           3.717627e+06  418143650.0   
           weasel_words    False          4.885422e+06  715353641.0   
                           True           7.660042e+05   74980034.0   

                                             clicks     purch  \
year_month feature         feature_value                        
2021-08    creative_type   carousel       2731361.0   36152.0   
                           dynamic        2175074.0   41627.0   
                           image          1294595.0   22434.0   
                           unknown        2306950.0   40886.0   
                           video          1101071.0   31390.0   
...                                             ...       ...   
2023-07    urgency         True           1707297.0   30406.0   
           user_addressing False          4291428.0   67180.0   
                           True           6660496.0  112670.0   
           weasel_words    False          9950143.0  161739.0   
                           True           1001781.0   18111.0   

                                          purch_value_USD  n_ads  \
year_month feature         feature_value                           
2021-08    creative_type   carousel          2.931933e+06   4930   
                           dynamic           2.838939e+06   9762   
                           image             1.328029e+06   7247   
                           unknown           2.091133e+06   8597   
                           video             1.556334e+06   4154   
...                                                   ...    ...   
2023-07    urgency         True              1.747118e+06   2382   
           user_addressing False             5.651549e+06   6218   
                           True              9.267915e+06  10103   
           weasel_words    False             1.369261e+07  13971   
                           True              1.226858e+06   2350   

                                          spend_USD_by_shop  impr_by_shop  \
year_month feature         feature_value                                    
2021-08    creative_type   carousel                0.262008      0.249020   
                           dynamic                 0.234237      0.241022   
                           image                   0.173454      0.175171   
                           unknown                 0.227982      0.235680   
                           video                   0.102319      0.099107   
...                                                     ...           ...   
2023-07    urgency         True                    0.123016      0.117647   
           user_addressing False                   0.517387      0.526622   
                           True                    0.482613      0.473378   
           weasel_words    False                   0.901555      0.898092   
                           True                    0.098445      0.101908   

                                          clicks_by_shop  purch_by_shop  \
year_month feature         feature_value                                  
2021-08    creative_type   carousel             0.256854       0.331802   
                           dynamic              0.237936       0.264744   
                           image                0.177017       0.145836   
             

In [81]:
n_shops_by_month_and_feature = relative_sum_by_value.reset_index().groupby(['year_month', 'feature'])['n_ads'].sum()

In [86]:
relative_sum

spend_USD      impr  \
shop_id   year_month feature         feature_value                        
2         2021-08    creative_type   carousel        0.238184  0.274308   
                                     dynamic         0.636367  0.529109   
                                     image           0.004610  0.003239   
                                     video           0.120839  0.193345   
                     cta             False           1.000000  1.000000   
...                                                       ...       ...   
135821055 2023-07    target          acquisition     1.000000  1.000000   
                     urgency         False           1.000000  1.000000   
                     user_addressing False           0.963964  0.980708   
                                     True            0.036036  0.019292   
                     weasel_words    False           1.000000  1.000000   

                                                      clicks     purch  \
shop_id   year_month feature         feature_value                       
2         2021-08    creative_type   carousel       0.314626  0.329970   
                                     dynamic        0.590841  0.590817   
                                     image          0.003731  0.006054   
                                     video          0.090802  0.073158   
                     cta             False          1.000000  1.000000   
...                                                      ...       ...   
135821055 2023-07    target          acquisition    1.000000  1.000000   
                     urgency         False          1.000000  1.000000   
                     user_addressing False          0.976217  0.963731   
                                     True           0.023783  0.036269   
                     weasel_words    False          1.000000  1.000000   

                                                    purch_value_USD     n_ads  
shop_id   year_month feature         feature_value                             
2         2021-08    creative_type   carousel              0.317657  0.182692  
                                     dynamic               0.602077  0.706731  
                                     image                 0.014612  0.028846  
                                     video                 0.065654  0.081731  
                     cta             False                 1.000000  1.000000  
...                                                             ...       ...  
135821055 2023-07    target          acquisition           1.000000  1.000000  
                     urgency         False                 1.000000  1.000000  
                     user_addressing False                 0.957599  0.916667  
                                     True                  0.042401  0.083333  
                     weasel_words    False                 1.000000  1.000000  

[298918 rows x 6 columns]

In [85]:
relative_sum / n_shops_by_month_and_feature

ValueError: cannot join with no overlapping index names

In [70]:
relative_sum

spend_USD        impr      clicks  \
year_month feature         feature_value                                       
2021-08    creative_type   carousel       137.816231  131.233651  133.306997   
                           dynamic        123.208917  127.018405  123.488860   
                           image           91.236672   92.314925   91.871912   
                           unknown        119.918445  124.203611  117.642960   
                           video           53.819735   52.229409   52.689270   
...                                              ...         ...         ...   
2023-07    urgency         True            53.511801   51.176310   50.696262   
           user_addressing False          225.063157  229.080567  225.872331   
                           True           209.936843  205.919433  205.127669   
           weasel_words    False          392.176559  390.670208  388.550778   
                           True            42.823441   44.329792   42.449222   

                                               purch  purch_value_USD  \
year_month feature         feature_value                                
2021-08    creative_type   carousel       130.398327       127.178601   
                           dynamic        104.044208       101.483269   
                           image           57.313580        53.986773   
                           unknown         59.682194        59.036732   
                           video           41.561691        42.314625   
...                                              ...              ...   
2023-07    urgency         True            49.721755        50.098164   
           user_addressing False          160.606120       160.695168   
                           True           160.393880       159.304832   
           weasel_words    False          290.348869       288.723133   
                           True            30.651131        31.276867   

                                               n_ads  
year_month feature         feature_value              
2021-08    creative_type   carousel       120.216309  
                           dynamic        124.673185  
                           image           98.655759  
                           unknown        128.104435  
                           video           57.350311  
...                                              ...  
2023-07    urgency         True            50.299489  
           user_addressing False          220.707686  
                           True           215.292314  
           weasel_words    False          394.168564  
                           True            41.831436  

[816 rows x 6 columns]

In [2]:
fcd.create_and_save_main(db=db, end_date=date.today(), force_from_scratch=True)

100%|██████████| 2/2 [00:16<00:00,  8.07s/it]


In [4]:
main_df = fcd.read_df('main', end_date=end_date)


,Unnamed: 0,year_month,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads,feature,shop_id
41,41,2023-06,False,136710.680000,9415026.0,83186.0,1112.0,181100.080000,20,starts_with_question,16038
42,42,2023-06,True,495.400000,17727.0,324.0,63.0,8721.330000,1,starts_with_question,16038
67,22,2023-06,False,137206.080000,9432753.0,83510.0,1175.0,189821.410000,21,cta,16038
97,28,2023-06,True,137206.080000,9432753.0,83510.0,1175.0,189821.410000,21,emoji,16038
124,25,2023-06,False,137206.080000,9432753.0,83510.0,1175.0,189821.410000,21,weasel_words,16038
150,24,2023-06,False,137206.080000,9432753.0,83510.0,1175.0,189821.410000,21,urgency,16038
184,32,2023-06,False,137206.080000,9432753.0,83510.0,1175.0,189821.410000,21,free_shipping,16038
215,29,2023-06,False,137206.080000,9432753.0,83510.0,1175.0,189821.410000,21,fact_words,16038
261,43,2023-06,False,70862.480000,5757317.0,51503.0,501.0,77518.980000,9,discount,16038
262,44,2023-06,True,66343.600000,3675436.0,32007.0,674.0,112302.430000,12,discount,16038


In [2]:
fcd.create_and_save_summary(end_date=end_date)

2023-08-21 12:49:52.905 | DEBUG    | src.services.descriptive.descriptive_saver:create_and_save_summary:36 - ['year_month', 'feature', 'feature_value']


In [2]:
summary_df = fcd.read_df('summary', end_date=end_date)
summary_df

,year_month,feature,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads,n_shops,spend_USD_by_shop,impr_by_shop,clicks_by_shop,purch_by_shop,purch_value_USD_by_shop,n_ads_by_shop
0,2021-08,creative_type,carousel,42030.279927,2720853.0,33467.0,502.0,90318.793686,45,2,0.794254,0.678537,0.782195,0.738235,0.796761,0.398230
1,2021-08,creative_type,dynamic,10887.664708,1289029.0,9319.0,178.0,23038.652777,68,2,0.205746,0.321463,0.217805,0.261765,0.203239,0.601770
2,2021-08,cta,False,52917.944635,4009882.0,42786.0,680.0,113357.446463,113,2,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
3,2021-08,discount,False,35690.936214,2774374.0,29747.0,455.0,78983.838743,39,2,0.674458,0.691884,0.695251,0.669118,0.696768,0.345133
4,2021-08,discount,True,17227.008421,1235508.0,13039.0,225.0,34373.607720,74,2,0.325542,0.308116,0.304749,0.330882,0.303232,0.654867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,2023-07,urgency,False,259475.009953,17869595.0,175611.0,3166.0,477205.144986,69,2,0.971241,0.926095,0.955399,0.881894,0.939683,0.945205
712,2023-07,urgency,True,7683.105525,1426041.0,8198.0,424.0,30631.421755,4,1,0.028759,0.073905,0.044601,0.118106,0.060317,0.054795
713,2023-07,user_addressing,False,9096.610000,598525.0,4972.0,42.0,7534.200000,3,1,0.034050,0.031019,0.027050,0.011699,0.014836,0.041096
714,2023-07,user_addressing,True,258061.505478,18697111.0,178837.0,3548.0,500302.366742,70,2,0.965950,0.968981,0.972950,0.988301,0.985164,0.958904


In [5]:
summary_df[summary_df.feature == 'creative_type'].groupby(['year_month', 'feature']).sum()

,,feature_value,spend_USD,impr,clicks,purch,purch_value_USD,n_ads,n_shops,spend_USD_by_shop,impr_by_shop,clicks_by_shop,purch_by_shop,purch_value_USD_by_shop,n_ads_by_shop
year_month,feature,,,,,,,,,,,,,,
2021-08,creative_type,carouseldynamic,52917.944635,4009882.0,42786.0,680.0,113357.446463,113,4,1.0,1.0,1.0,1.0,1.0,1.0
2021-09,creative_type,carouseldynamic,24165.890637,1971449.0,17117.0,630.0,63470.895325,66,3,1.0,1.0,1.0,1.0,1.0,1.0
2021-10,creative_type,carouseldynamic,26427.459235,2223481.0,20100.0,824.0,87163.549962,82,3,1.0,1.0,1.0,1.0,1.0,1.0
2021-11,creative_type,carouseldynamic,52778.190299,3587435.0,31512.0,2046.0,237975.227778,98,4,1.0,1.0,1.0,1.0,1.0,1.0
2021-12,creative_type,carouseldynamicimageunknown,46631.926584,3184183.0,35885.0,876.0,105553.249608,178,6,1.0,1.0,1.0,1.0,1.0,1.0
2022-01,creative_type,carouseldynamicimageunknownvideo,137052.389716,9453957.0,102873.0,1268.0,239731.618990,121,7,1.0,1.0,1.0,1.0,1.0,1.0
2022-02,creative_type,carouseldynamicimagevideo,195549.138820,9303403.0,90777.0,1487.0,288846.630864,65,7,1.0,1.0,1.0,1.0,1.0,1.0
2022-03,creative_type,carouseldynamicimagevideo,277006.652355,14503907.0,134153.0,4358.0,598468.803013,312,7,1.0,1.0,1.0,1.0,1.0,1.0
2022-04,creative_type,carouseldynamicimagevideo,89604.532922,3828222.0,42996.0,1005.0,168579.315022,165,7,1.0,1.0,1.0,1.0,1.0,1.0


In [4]:
main_df = fcd.get_main_df(end_date=end_date)
main_df

spend_USD         impr  \
year_month feature         feature_value                              
2021-08    creative_type   carousel       1.385117e+06  189679520.0   
                           dynamic        2.015207e+06  231822297.0   
                           image          1.122925e+06  179716465.0   
                           unknown        1.297635e+06  193497346.0   
                           video          1.305978e+06   95865870.0   
...                                                ...          ...   
2023-07    urgency         True           6.187278e+05  110600249.0   
           user_addressing False          1.931415e+06  371170936.0   
                           True           3.717662e+06  418116014.0   
           weasel_words    False          4.883056e+06  714306181.0   
                           True           7.660223e+05   74980769.0   

                                             clicks     purch  \
year_month feature         feature_value                        
2021-08    creative_type   carousel       2731351.0   36151.0   
                           dynamic        2174472.0   41599.0   
                           image          1288717.0   22366.0   
                           unknown        2302229.0   40817.0   
                           video          1101071.0   31390.0   
...                                             ...       ...   
2023-07    urgency         True           1707297.0   30406.0   
           user_addressing False          4285518.0   67128.0   
                           True           6660266.0  112665.0   
           weasel_words    False          9943996.0  161682.0   
                           True           1001788.0   18111.0   

                                          purch_value_USD    n_ads  \
year_month feature         feature_value                             
2021-08    creative_type   carousel          2.932261e+06   4929.0   
                           dynamic           2.836585e+06   9757.0   
                           image             1.323270e+06   7237.0   
                           unknown           2.087038e+06   8565.0   
                           video             1.556321e+06   4154.0   
...                                                   ...      ...   
2023-07    urgency         True              1.747109e+06   2382.0   
           user_addressing False             5.646147e+06   6168.0   
                           True              9.267490e+06  10101.0   
           weasel_words    False             1.368683e+07  13919.0   
                           True              1.226810e+06   2350.0   

                                          spend_USD_by_shop  impr_by_shop  \
year_month feature         feature_value                                    
2021-08    creative_type   carousel              137.795773    131.217613   
                           dynamic               122.230413    126.040613   
                           image                  90.810532     91.908048   
                           unknown               118.337687    122.586704   
                           video                  53.825595     52.247022   
...                                                     ...           ...   
2023-07    urgency         True                   53.511801     51.176310   
           user_addressing False                 220.288345    224.220472   
                           True                  209.711655    205.779528   
           weasel_words    False                 387.141526    385.659054   
                           True                   42.858474     44.340946   

                                          clicks_by_shop  purch_by_shop  \
year_month feature         feature_value                                  
2021-08    creative_type   carousel           133.292906     130.363844   
                           dynamic            122.505200     103.078691   
                           image               91.310972      

In [5]:
set(main_df.index.get_level_values(level='feature').tolist())

{'creative_type',
 'cta',
 'discount',
 'emoji',
 'fact_words',
 'fake_column_for_total_sum',
 'free_shipping',
 'hashtag',
 'link',
 'percentage',
 'price',
 'starts_with_question',
 'target',
 'urgency',
 'user_addressing',
 'weasel_words'}

In [6]:
main_df.reset_index(inplace=True)

In [8]:
df_fw = fcd.get_shop_descriptive_df(db=db, shop_id=16038, end_date=end_date, start_date = start_date)
df_dr = fcd.get_shop_descriptive_df(db=db, shop_id=44301396, end_date=end_date, start_date = start_date)
df_test = df_fw.add(df_dr, fill_value=0)
df_test

2023-08-17 11:13:28.909 | DEBUG    | src.services.descriptive.descriptive_saver:get_shop_descriptive_df:115 - 16038: +
2023-08-17 11:13:34.219 | DEBUG    | src.services.descriptive.descriptive_saver:get_shop_descriptive_df:115 - 44301396: +


spend_USD        impr    clicks  \
year_month feature         feature_value                                        
2021-08    creative_type   carousel        42026.960098   2720853.0   33467.0   
                           dynamic         10883.048792   1289029.0    9319.0   
           cta             False           52910.008889   4009882.0   42786.0   
           discount        False           35686.750467   2774374.0   29747.0   
                           True            17223.258422   1235508.0   13039.0   
...                                                 ...         ...       ...   
2023-07    urgency         False          259459.083828  17869595.0  175611.0   
                           True             7673.135233   1426041.0    8198.0   
           user_addressing False            9096.610000    598525.0    4972.0   
                           True           258035.609061  18697111.0  178837.0   
           weasel_words    False          267132.219061  19295636.0  183809.0   

                                           purch  purch_value_USD  n_ads  \
year_month feature         feature_value                                   
2021-08    creative_type   carousel        502.0     90307.824811   45.0   
                           dynamic         178.0     23024.805496   68.0   
           cta             False           680.0    113332.630308  113.0   
           discount        False           455.0     78971.516448   39.0   
                           True            225.0     34361.113859   74.0   
...                                          ...              ...    ...   
2023-07    urgency         False          3166.0    477140.981293   69.0   
                           True            424.0     30591.671656    4.0   
           user_addressing False            42.0      7534.200000    3.0   
                           True           3548.0    500198.452949   70.0   
           weasel_words    False          3590.0    507732.652949   73.0   

                                          spend_USD_by_shop  impr_by_shop  \
year_month feature         feature_value                                    
2021-08    creative_type   carousel                1.261710      1.180138   
                           dynamic                 0.738290      0.819862   
           cta             False                   2.000000      2.000000   
           discount        False                   1.221120      1.279078   
                           True                    0.778880      0.720922   
...                                                     ...           ...   
2023-07    urgency         False                   1.614993      1.602622   
                           True                    0.385007      0.397378   
           user_addressing False                   0.036798      0.038106   
                           True                    1.963202      1.961894   
           weasel_words    False                   2.000000      2.000000   

                                          clicks_by_shop  purch_by_shop  \
year_month feature         feature_value                                  
2021-08    creative_type   carousel             1.273955       1.291995   
                           dynamic              0.726045       0.708005   
           cta             False                2.000000       2.000000   
           discount        False                1.248224       1.260297   
                           True                 0.751776       0.739703   
...                                                  ...            ...   
2023-07    urgency         False                1.657332       1.648425   
                           True                 0.342668       0.351575   
           user_addressing False                0.031097       0.017617   
                           True                 1.968903       1.982383   
           weasel_words    False                2.000000       2.000000   

                         

In [9]:
main_df.subtract(df_test)

spend_USD  impr  clicks  purch  \
year_month feature         feature_value                                   
2021-08    creative_type   carousel             0.0   0.0     0.0    0.0   
                           dynamic              0.0   0.0     0.0    0.0   
           cta             False                0.0   0.0     0.0    0.0   
           discount        False                0.0   0.0     0.0    0.0   
                           True                 0.0   0.0     0.0    0.0   
...                                             ...   ...     ...    ...   
2023-07    urgency         False                0.0   0.0     0.0    0.0   
                           True                 0.0   0.0     0.0    0.0   
           user_addressing False                0.0   0.0     0.0    0.0   
                           True                 0.0   0.0     0.0    0.0   
           weasel_words    False                0.0   0.0     0.0    0.0   

                                          purch_value_USD  n_ads  \
year_month feature         feature_value                           
2021-08    creative_type   carousel          0.000000e+00    0.0   
                           dynamic           0.000000e+00    0.0   
           cta             False             0.000000e+00    0.0   
           discount        False             0.000000e+00    0.0   
                           True              0.000000e+00    0.0   
...                                                   ...    ...   
2023-07    urgency         False             0.000000e+00    0.0   
                           True              0.000000e+00    0.0   
           user_addressing False             0.000000e+00    0.0   
                           True              5.820766e-11    0.0   
           weasel_words    False             0.000000e+00    0.0   

                                          spend_USD_by_shop  impr_by_shop  \
year_month feature         feature_value                                    
2021-08    creative_type   carousel            0.000000e+00  0.000000e+00   
                           dynamic             0.000000e+00  0.000000e+00   
           cta             False               0.000000e+00  0.000000e+00   
           discount        False               0.000000e+00  2.220446e-16   
                           True                0.000000e+00  0.000000e+00   
...                                                     ...           ...   
2023-07    urgency         False               0.000000e+00  0.000000e+00   
                           True                0.000000e+00  0.000000e+00   
           user_addressing False              -2.081668e-17 -1.387779e-17   
                           True                0.000000e+00  0.000000e+00   
           weasel_words    False               0.000000e+00  0.000000e+00   

                                          clicks_by_shop  purch_by_shop  \
year_month feature         feature_value                                  
2021-08    creative_type   carousel        -2.220446e-16   0.000000e+00   
                           dynamic          0.000000e+00   0.000000e+00   
           cta             False            0.000000e+00   0.000000e+00   
           discount        False            0.000000e+00   0.000000e+00   
                           True             0.000000e+00   0.000000e+00   
...                                                  ...            ...   
2023-07    urgency         False            0.000000e+00   0.000000e+00   
                           True            -5.551115e-17   0.000000e+00   
           user_addressing False           -3.122502e-17  -2.775558e-17   
                           True            -2.220446e-16   0.000000e+00   
           weasel_words    False            0.000000e+00   0.000000e+00   

                                          purch_value_USD_by_shop  \
year_month feature         feature_value                            
2021-08    creative_type   carousel                 